*ETL*

Cargar todas las librerías a utilizar:

In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import json
import ast
import pyarrow
import uvicorn
import os
import pyarrow

Importar el CSV a Visual Code a través de Python de manera adecuada para que funcione en distintos entornos

In [27]:
base_dir = os.getcwd()
file_path = os.path.join(base_dir, '..', 'Datasets', 'movies_FINAL.csv')
df = pd.read_csv(file_path)


Revisar el tipo de dato en cada columna del DataFrame para seleccionar cuáles son relevantes para el proyecto

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   adult                     45463 non-null  bool   
 1   budget                    45463 non-null  int64  
 2   homepage                  7779 non-null   object 
 3   movie_id                  45463 non-null  int64  
 4   imdb_id                   45446 non-null  object 
 5   original_language         45452 non-null  object 
 6   original_title            45463 non-null  object 
 7   overview                  44509 non-null  object 
 8   popularity                45463 non-null  float64
 9   release_date              45379 non-null  object 
 10  revenue                   45463 non-null  int64  
 11  runtime                   45206 non-null  float64
 12  status                    45382 non-null  object 
 13  title                     45463 non-null  object 
 14  video 

Se pasa a int la columna budget y la columna ID que servirán para las futuras funciones.

In [29]:
# Reemplazar el valor '1997-08-20' con 0
df['budget'] = df['budget'].replace(pd.Timestamp('1997-08-20'), 0)

# Convertir la columna 'budget' a tipo int
df['budget'] = df['budget'].astype(int, errors='ignore')




In [30]:
# Reemplazar el valor '1997-08-20' con 0
df['movie_id'] = df['movie_id'].replace(pd.Timestamp('1997-08-20'), 0)

# Reemplazar NaT con 0
df['movie_id'] = df['movie_id'].fillna(0)


# Convertir la columna 'id' a tipo int
df['movie_id'] = df['movie_id'].astype(int, errors='ignore')

Eliminar video,imdb_id,adult,original_title,poster_path y homepage.


In [35]:
df=df.drop(columns=["video","imdb_id","adult","original_title","homepage"])

Identificar las variables que tienen datos anidados para darles el correcto tratamiento:
- belongs_to_collection
- genres
- production_companies
- production_coutries
- spoken_languages


In [31]:
def extract_full_collection_info(row):
    if pd.isna(row['belongs_to_collection']):
        return pd.Series([None, None, None, None], 
                         index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])
    else:
        try:
            collection = json.loads(row['belongs_to_collection'].replace("'", "\""))
            return pd.Series([collection.get('id'), collection.get('name'), collection.get('poster_path'), collection.get('backdrop_path')], 
                             index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])
        except:
            return pd.Series([None, None, None, None], 
                             index=['collection_id', 'collection_name', 'collection_poster_path', 'collection_backdrop_path'])

# Aplicar la función a cada fila del DataFrame
full_collection_info = df.apply(extract_full_collection_info, axis=1)

# Añadir las nuevas columnas al DataFrame original
aux = pd.concat([df, full_collection_info],axis=1)

KeyError: 'belongs_to_collection'

In [ ]:

def genres_ids(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['id'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def genres_names(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['genres_ids'] = df['genres'].apply(genres_ids)
df['genres_names'] = df['genres'].apply(genres_names)

df['genres_names_array'] = df['genres_names'].apply(lambda x: x if isinstance(x,list)else[])
aux5 = df[['genres_names_array']]                                                                              


In [ ]:
aux5

,genres_names_array
0,"[Animation, Comedy, Family]"
1,"[Adventure, Fantasy, Family]"
2,"[Romance, Comedy]"
3,"[Comedy, Drama, Romance]"
4,[Comedy]
...,...
45458,"[Drama, Family]"
45459,[Drama]
45460,"[Action, Drama, Thriller]"
45461,[]


In [18]:

def extract_production_company_ids(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['id'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_production_company_names(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['production_company_ids'] = df['production_companies'].apply(extract_production_company_ids)
df['production_company_names'] = df['production_companies'].apply(extract_production_company_names)

df['company_names_array'] = df['production_company_names'].apply(lambda x: x if isinstance(x,list)else[])
aux2 = df[['company_names_array']]                                                                              




KeyError: 'production_companies'

In [19]:
aux2

NameError: name 'aux2' is not defined

In [21]:

def extract_production_company_iso(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['iso_3166_1'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_production_company_paises(companies):
    try:
        companies_list = ast.literal_eval(companies)
        return [company['name'] for company in companies_list]
    except (ValueError, SyntaxError, TypeError):
        return []

df['production_countries_iso'] = df['production_countries'].apply(extract_production_company_iso)
df['production_countries_names'] = df['production_countries'].apply(extract_production_company_paises)

df['countries_names_array'] = df['production_countries_names'].apply(lambda x: x if isinstance(x,list)else[])
aux3 = df[['countries_names_array']]    



KeyError: 'production_countries'

In [22]:
aux3

NameError: name 'aux3' is not defined

In [23]:
def extract_iso_languages(languages):
    try:
        languages_list = ast.literal_eval(languages)
        return [lang['iso_639_1'] for lang in languages_list]
    except (ValueError, SyntaxError, TypeError):
        return []

def extract_language_names(languages):
    try:
        languages_list = ast.literal_eval(languages)
        return [lang['name'] for lang in languages_list]
    except (ValueError, SyntaxError, TypeError):
        return []


# Aplicar las funciones a la columna 'spoken_languages'
df['spoken_languages_iso'] = df['spoken_languages'].apply(extract_iso_languages)
df['spoken_languages_name'] = df['spoken_languages'].apply(extract_language_names)

df['languages_names_array'] = df['spoken_languages_name'].apply(lambda x: x if isinstance(x,list)else[])
aux4 = df[['languages_names_array']]  

KeyError: 'spoken_languages'

In [24]:
aux4

NameError: name 'aux4' is not defined

Uno los Dataframe nuevos al original

In [17]:
df_final = pd.concat([aux, aux2, aux3, aux4, aux5],axis=1)


NameError: name 'aux' is not defined

In [ ]:
df_final.shape

(45463, 32)

Se eliminan todas las columnas que no presentan utilidad para el posterior proceso de recomendación.

In [32]:
df_final=df_final.drop(columns=["belongs_to_collection","production_companies","production_countries","spoken_languages","genres","poster_path","tagline","collection_id","collection_poster_path"])

NameError: name 'df_final' is not defined

Se reliza la conversión de ID y Budget para  utilizar posteriormente

In [ ]:
df_final.rename(columns={'id':'movie_id'},inplace=True)

In [ ]:
df_final['movie_id'] = df_final['movie_id'].fillna(0).astype(int)

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   adult                     45463 non-null  bool   
 1   budget                    45463 non-null  int32  
 2   homepage                  7779 non-null   object 
 3   movie_id                  45463 non-null  int32  
 4   imdb_id                   45446 non-null  object 
 5   original_language         45452 non-null  object 
 6   original_title            45463 non-null  object 
 7   overview                  44509 non-null  object 
 8   popularity                45463 non-null  float64
 9   release_date              45379 non-null  object 
 10  revenue                   45463 non-null  int64  
 11  runtime                   45206 non-null  float64
 12  status                    45382 non-null  object 
 13  title                     45463 non-null  object 
 14  video 

Se trasnforma en CSV para poder desarrollar una API luego.

In [16]:
df_final.to_csv(r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\movies_output.csv', index= False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\jessi\\Desktop\\Henry\\PROYECTO FINAL INDIVIDUAL\\Datasets\\movies_output.csv'

Se realiza el ETL del archivo credits.csv

In [36]:
df_credits = pd.read_parquet('Datasets/cast.parquet')
df_credits2 = pd.read_parquet('Datasets/crew.parquet')


FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/cast.parquet'

Se separa en dos el dataframe

In [ ]:
df_credit_cast = df_credits[['cast', 'id']].copy()

df_credit_crew = df_credits[['crew','id']].copy()

Se cambia el nombre a movie_id para facilitar la comprensión del código. Luego se parsean las columnas anidadas

In [ ]:
df_credit_cast.rename(columns={'id':'movie_id'},inplace=True)

In [ ]:
df_credit_crew.rename(columns={'id':'movie_id'},inplace=True)

In [ ]:
 
def expand_columns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        df = df.explode(column)
        col_df = pd.json_normalize(df[column])
        col_df = col_df.add_prefix(f'{column}_')
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['cast']

df_credit_cast = expand_columns(df_credit_cast, columns_to_expand)

In [ ]:
columns_to_expand = ['crew']
df_credit_crew = expand_columns(df_credit_crew, columns_to_expand)


Se eliminan columnas para poder crear un API luego.

In [ ]:
df_credit_cast = df_credit_cast.drop(columns=["cast_cast_id","cast_credit_id","cast_id","cast_profile_path"])
#se eliminaron en total credit_id, order, profile_path y cast_credit_id


In [ ]:
df_credit_crew = df_credit_crew.drop(columns=["crew_id"])


In [ ]:
df_credit_cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564892 entries, 0 to 564891
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   movie_id        564892 non-null  int64  
 1   cast_character  562474 non-null  object 
 2   cast_gender     562474 non-null  float64
 3   cast_name       562474 non-null  object 
 4   cast_order      562474 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 21.5+ MB


In [ ]:
df_credit_crew.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465085 entries, 0 to 465084
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           465085 non-null  int64  
 1   crew_credit_id     464314 non-null  object 
 2   crew_department    464314 non-null  object 
 3   crew_gender        464314 non-null  float64
 4   crew_job           464314 non-null  object 
 5   crew_name          464314 non-null  object 
 6   crew_profile_path  95098 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 24.8+ MB


In [ ]:
crew_cast.to_parquet(r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\cast_crew.parquet')

In [ ]:

output_file_path = r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\cast.parquet'
output_file_path1 = r'C:\Users\jessi\Desktop\Henry\PROYECTO FINAL INDIVIDUAL\Datasets\crew.parquet'
df_credit_cast.to_parquet(output_file_path)
df_credit_crew.to_parquet(output_file_path1)

Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0. Dataset: Movies

In [ ]:
df_final['revenue'].fillna(0)
df_final['budget'].fillna(0)

0        30000000
1        65000000
2               0
3        16000000
4               0
           ...   
45458           0
45459           0
45460           0
45461           0
45462           0
Name: budget, Length: 45463, dtype: int32

Los valores nulos del campo release date deben eliminarse.


In [ ]:
df_final['release_date'].dropna()



0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45457    1991-05-13
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45379, dtype: object

Convertir en tipo fecha los datos de la columna Date_Release con correspondiente manejo de errores

In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45463 entries, 0 to 45462
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   adult                     45463 non-null  bool   
 1   budget                    45463 non-null  int32  
 2   homepage                  7779 non-null   object 
 3   movie_id                  45463 non-null  int32  
 4   imdb_id                   45446 non-null  object 
 5   original_language         45452 non-null  object 
 6   original_title            45463 non-null  object 
 7   overview                  44509 non-null  object 
 8   popularity                45463 non-null  float64
 9   release_date              45379 non-null  object 
 10  revenue                   45463 non-null  int64  
 11  runtime                   45206 non-null  float64
 12  status                    45382 non-null  object 
 13  title                     45463 non-null  object 
 14  video 

In [ ]:
df_final['release_date'].fillna('1900-01-01')


0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45458    1900-01-01
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45463, dtype: object

In [ ]:
df_final['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45458           NaN
45459    2011-11-17
45460    2003-08-01
45461    1917-10-21
45462    2017-06-09
Name: release_date, Length: 45463, dtype: object

In [ ]:
df_final['release_date'] = pd.to_datetime(df_final['release_date'], errors='coerce')

Generar una nueva columna llamada "release_year" en donde figurará el año de estreno de la película.

In [ ]:
df_final['release_year']=df_final['release_date'].dt.year


Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.



In [15]:
df_final['budget'] = pd.to_numeric(df_final['budget'], errors='coerce').astype('Int64')


NameError: name 'df_final' is not defined

Cada columna tiene sus NaN reemplazados por ceros.

In [ ]:
df_final['revenue'].fillna(0, inplace=True)
df_final['budget'].fillna(0, inplace=True)

Se realiza la cuenta para generar el KPI y se reemplazan los "inf" y "NaN" con ceros.

In [ ]:
df_final['return'] = df_final['revenue'] / df_final['budget']
df_final['return'] = df_final['return'].replace(np.inf, 0).astype(float)
df_final['return'].fillna(0,inplace=True)

